## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import Google_Map_API_KEY

# Configure gmaps
gmaps.configure(api_key=Google_Map_API_KEY)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Mataura,NZ,57.47,few clouds,-46.1927,168.8643,Ellie's Villa
1,3,Brae,GB,39.11,overcast clouds,60.3964,-1.3530,Busta House Hotel
2,7,Komsomolskiy,UZ,38.12,overcast clouds,40.4272,71.7189,Ikathouse
3,8,Matamba,TZ,55.56,overcast clouds,-8.9833,33.9667,Lena blumen lodge
4,19,Huilong,CN,41.99,overcast clouds,31.8111,121.6550,Hilton Garden Inn Qidong


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

# City Start: Riesa, DE(Germany)
# stop 1: Hof, DE(Germany)
# stop 2: Mainburg, DE(Germany)
# stop 3: Zirovnice, CZ(Czechia) 
# City end: (same as start) Riesa, DE(Germany)

# located city AND country (incase more than one city with same city name)
vacation_start = vacation_df.loc[(vacation_df["City"] == 'Riesa') & (vacation_df["Country"] == 'DE')]
display(vacation_start)
vacation_end = vacation_df.loc[(vacation_df["City"] == 'Riesa') & (vacation_df["Country"] == 'DE')]
display(vacation_end)
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == 'Hof') & (vacation_df["Country"] == 'DE')]
display(vacation_stop1)
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == 'Mainburg') & (vacation_df["Country"] == 'DE')] 
display(vacation_stop2)
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == 'Zirovnice') & (vacation_df["Country"] == 'CZ')] 
display(vacation_stop3)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
76,243,Riesa,DE,43.12,light rain,51.3,13.3,Mercure Hotel Riesa Dresden Elbland


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
76,243,Riesa,DE,43.12,light rain,51.3,13.3,Mercure Hotel Riesa Dresden Elbland


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
189,573,Hof,DE,38.44,overcast clouds,50.3167,11.9167,Hotel Burghof


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
162,496,Mainburg,DE,37.24,overcast clouds,48.65,11.7833,Heimat Mainburg | Hotel • Café • Weinbar


,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
229,678,Zirovnice,CZ,36.82,broken clouds,49.2532,15.1882,Hotel Artaban


In [10]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
# Reference: https://stackoverflow.com/questions/10016352/convert-numpy-array-to-tuple
# I was only able to get to_numpy() output such as "array([[45.1667, 10,7833]])"
# But when we use that to fill in the directions_layer,
# it given error "[[45.1667 10.7833]] is not a valid location. Locations must have length 2."
# So I searched and figure that we want output as tuple (45.1667,10,7833), and here's the link I found and helped me solve
# reference: https://stackoverflow.com/questions/10016352/convert-numpy-array-to-tuple

# tuple(numpy array [0])
# [0]: read the value at index 0, if not adding [0], it will print the whole thing included "array[]"

start = tuple(vacation_start[['Lat','Lng']].to_numpy()[0])
end = tuple(vacation_end[['Lat','Lng']].to_numpy()[0])
stop1 = tuple(vacation_stop1[['Lat','Lng']].to_numpy()[0])
stop2 = tuple(vacation_stop2[['Lat','Lng']].to_numpy()[0])
stop3 = tuple(vacation_stop3[['Lat','Lng']].to_numpy()[0])

In [11]:
display(start)
display(end)
display(stop1)
display(stop2)
display(stop3)


(51.3, 13.3)

(51.3, 13.3)

(50.3167, 11.9167)

(48.65, 11.7833)

(49.2532, 15.1882)

In [12]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
direction_map = gmaps.directions_layer(start, end, waypoints=[stop1,stop2,stop3], travel_mode = 'DRIVING')
fig.add_layer(direction_map)
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_end, vacation_stop1, vacation_stop2, vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,243,Riesa,DE,43.12,light rain,51.3000,13.3000,Mercure Hotel Riesa Dresden Elbland
1,243,Riesa,DE,43.12,light rain,51.3000,13.3000,Mercure Hotel Riesa Dresden Elbland
2,573,Hof,DE,38.44,overcast clouds,50.3167,11.9167,Hotel Burghof
3,496,Mainburg,DE,37.24,overcast clouds,48.6500,11.7833,Heimat Mainburg | Hotel • Café • Weinbar
4,678,Zirovnice,CZ,36.82,broken clouds,49.2532,15.1882,Hotel Artaban


In [14]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))